# Tutorial Básico de Criptografía Homomórfica

En este tutorial básico vamos a ver como se pueden cifrar datos con esquemas de criptografía homomórfica, realizar operaciones y comprobar que el resultado es igual que si las operaciones se hubiesen hecho con los datos sin cifrar.

In [1]:
import tenseal as ts
import random

# Elementos Auxiliares

Para este tutorial vamos a necesitar importar las librearías de TenSEAL y random (generación de números aleatorios). Además nos definimos dos funciones que serán útiles más adelante en el tutorial.

In [2]:
# Método que nos genera un vector de tamaño size con números aleatorios hasta el 4
def generate_vector(size):
    return [random.randint(0, 4) for _ in range(size)]

# Nos calcula la diferencia media entre elementos de una lista
def average_difference_between_lists(list1, list2):
    if len(list1) != len(list2):
        raise ValueError("Lists must be of the same size.")

    differences = [abs(list1[i] - list2[i]) for i in range(len(list1))]
    avg_diff = sum(differences) / len(differences)
    return avg_diff

# Contextos

Uno de los objetos más importantes en TenSEAL es el contexto. Este objeto guarda información relevante como las claves de cifrado y los parámetros del esquema de cifrado homórfico utilizado. De esta manera sólo hará falta utilizar este objeto para la mayoría de operaciones necesarias. En este caso vamos a crear un contexto para empezar a realizar nuestras operaciones.

In [3]:
# Creación del contexto
# Controla la precisión de la parte decimal
bits_scale = 26

# Creación del contexto en TenSEAL. Utilizamos CKKS que permite realizar cómputos con números reales
# CKKS fue presentado en ASCIACCS 17: https://eprint.iacr.org/2016/421.pdf
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[31, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, bits_scale, 31]
)
# Parámetros adicionales
context.global_scale = pow(2, bits_scale)
context.generate_galois_keys()

# Cifrando un Vector

El siguiente código cifra un vector simple

In [4]:
plain_vector = [60, 66, 73, 81, 90]
encrypted_vector = ts.ckks_vector(context, plain_vector)
print("Tamaño del vector cifrado:", encrypted_vector.size())
encrypted_vector

Tamaño del vector cifrado: 5


Ahora vamos a proceder a realizar una suma

In [5]:
add_result = encrypted_vector + [1, 2, 3, 4, 5]
print(add_result.decrypt())

[61.00000419242638, 67.99996391342151, 75.99997826289973, 85.00002251947531, 94.9999963398753]


¿Qué valores obtenemos?

Con la multiplicación pasa algo similar

In [6]:
mul_result = encrypted_vector * [1, 2, 3, 4, 5]
print(mul_result.decrypt())

[60.05863741151413, 132.12895145174554, 219.21406665988422, 324.3168189044186, 450.4398309917207]


Ahora vamos a realizar operaciones y observar como el error se va acumulando hasta que llega a un límite definido por nosotros. El código proporcionado muestra como el 'error' se va acumulando según se siguen realizando operaciones con los datos. Los algoritmos de criptografía homomórfica tienen mecanismos para ir reduciendo este 'error' pero hacen que la computación sea muy ineficiente comparada con los cómputos sobre datos sin cifrar.

In [7]:
# Generamos un vector aleatorio
vector_size = 5
original_vector = generate_vector(vector_size)
print("Vector Original:", original_vector)
# Ciframos el vector
encrypted_vector = ts.ckks_vector(context, original_vector)
decrypted_result = []
original_result = original_vector
max_avg_error = 10
# Realizamos operaciones hasta llegar a un error máximo
for i in range(1000):
    # Multiplicamos el vector cifrado
    encrypted_vector = encrypted_vector * [1, 2, 3, 4, 5]
    # Multiplicamos el vector original
    original_result = [a * b for a, b in zip(original_result, [1, 2, 3, 4, 5])]
    # Desciframos el resultado y mostramos
    decrypted_result = encrypted_vector.decrypt()
    print("Resultado Descifrado:", decrypted_result)
    print("Resultado Original:", original_result)
    # Si la diferencia supera un límite salimos
    print("Diferencia media: ", average_difference_between_lists(original_result,decrypted_result))
    if average_difference_between_lists(original_result,decrypted_result)>max_avg_error:
        print("Salimos en la iteración", i)
        break

Vector Original: [3, 3, 0, 3, 1]
Resultado Descifrado: [3.002946720716774, 6.005894568340625, 3.6348748826789454e-05, 12.011734826461126, 5.00508642875416]
Resultado Original: [3, 6, 0, 12, 5]
Diferencia media:  0.005139778604302224
Resultado Descifrado: [3.00807387889559, 12.032334137434756, 9.356161870050319e-05, 48.129179523782106, 25.068262020520685]
Resultado Original: [3, 12, 0, 48, 25]
Diferencia media:  0.04758862445036741
Resultado Descifrado: [3.0146772260690895, 24.11766744909388, 0.00027001949163233275, 192.94069815021498, 125.61731836545317]
Resultado Original: [3, 24, 0, 192, 125]
Diferencia media:  0.3381262420645517
Resultado Descifrado: [3.0279568907940915, 48.448275248935985, 0.0008360955489383527, 775.1693383527777, 630.858875338803]
Resultado Original: [3, 48, 0, 768, 625]
Diferencia media:  2.7010563853719494
Resultado Descifrado: [3.0533290340872252, 97.70758930354401, 0.0025525514749779312, 3126.631019318586, 3180.696427246417]
Resultado Original: [3, 96, 0, 3072

# Tarea: Puedes resolver el problema de los millonarios con criptografía homomórfica?

**Contexto del problema:**

saber quién es el más rico entre dos millonarios sin que ninguno de ellos sepa cuánto tiene el otro.

**Solución:**

cifrar los datos de ambos millonarios y hacer la resta.